# Install or upgreade an required package into notebook environment

In [ ]:
!pip install abacusai --upgrade

# Import Packages required
## We will be using pandas and the Abacus.ai client

In [ ]:
import pandas as pd
from abacusai import ApiClient, ApiException

# Build a local model
To begin, we'll read a dataset from s3 into a pandas dataframe. We then manipulate the data, prepare it for training and train a model locally

In [ ]:
concrete_df = pd.read_csv(
    "s3://abacusai-exampledatasets/predicting/concrete_measurements.csv"
)
concrete_df.head()

### We perform a simple transform. Please be sure to provide a constant we utilize in our transform function
Importing pandas directly in the function body is important for later. Be sure to test your transform function!

In [ ]:
def transform_concrete(concrete_dataframe):
    import pandas as pd

    constant = None
    assert constant, "Please provide a constant to use in featurization"
    # Remove the flyash feature
    feature_df = concrete_dataframe.drop(["flyash"], axis=1).copy()
    # DataFrame with rows where flyash == 0
    no_flyash = feature_df[concrete_dataframe["flyash"] == 0.0].copy()
    no_flyash.loc[:, "slag"] = no_flyash["slag"] - no_flyash["water"].mean()
    # DataFrame with rows where flyash > 0
    flyash = feature_df[concrete_dataframe["flyash"] > 0.0].copy()
    flyash.loc[:, "slag"] = flyash["slag"] + constant
    return pd.concat([no_flyash, flyash - flyash.assign(age=0).mean()])

In [ ]:
transformed_concrete_df = transform_concrete(concrete_df)
transformed_concrete_df

### We now prepare the data for training by splitting it between a train and test split

In [ ]:
def assign_train_test_split(data_to_split):
    import pandas as pd
    from sklearn.model_selection import train_test_split

    training_data, prediction_data = train_test_split(
        data_to_split.copy(), test_size=0.1, train_size=0.9, random_state=42
    )
    training_data["TRAINTEST"] = "TRAIN"
    prediction_data["TRAINTEST"] = "TEST"
    return pd.concat([training_data, prediction_data])

In [ ]:
concrete_train_test = assign_train_test_split(transformed_concrete_df)
concrete_training_data = concrete_train_test.loc[
    concrete_train_test["TRAINTEST"] == "TRAIN"
].drop(columns={"TRAINTEST"}, axis=1)
concrete_prediction_data = concrete_train_test.loc[
    concrete_train_test["TRAINTEST"] == "TEST"
].drop(columns={"TRAINTEST"}, axis=1)

## Define a local training and predict function
We define a simple linear regression model that utilizes a quantile transformer for numeric columns. We also define a prediction function. Of course, we execute both of these to ensure they work.

In [ ]:
def train(training_dataset):
    import numpy as np

    # set the seed for reproduceable results
    np.random.seed(5)

    target_column = "csMPa"
    X = training_dataset.drop([target_column], axis=1).copy()
    y = training_dataset[target_column]
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import QuantileTransformer

    qt = QuantileTransformer(n_quantiles=20)
    linear_regresion = LinearRegression()
    linear_regresion.fit(qt.fit_transform(X), y)
    print(f"Linear model R^2 = {linear_regresion.score(qt.transform(X), y)}")

    return (X.columns, target_column, qt, linear_regresion)

### Examine the train function outputs

In [ ]:
local_model = train(concrete_training_data)
input_columns, target, transform, model = local_model
print(input_columns)
print(target)
print(transform)
print(model)

In [ ]:
def predict_many(model, queries):
    input_columns, target_column, qt, model = model
    import pandas as pd

    predictions = []
    for query in queries:
        X = pd.DataFrame({c: [query[c]] for c in input_columns})
        y = model.predict(qt.transform(X))
        predictions.append({target_column: y[0]})
    return predictions

### Let's see what our model is predicting
We aren't too worried about performance for our toy model

In [ ]:
prediction_results = predict_many(
    local_model, concrete_prediction_data.to_dict(orient="records")
)
prediction_results[:5]

# Incorporate our Model into Abacus
With a working local model, we can now leverage the Abacus.ai client and our training/predict code to bring our model into the Abacus Platform

In [ ]:
# Instatiate a client instance
client = ApiClient(server='https://workshop.abacus.ai')

## Creating our first project
The first project we create is a Feature Store project. This will act as a location to store feature groups (data transformations) we create.
https://workshop.abacus.ai/help/useCases/feature_store

In [ ]:
feature_store_project_name = ""
assert (
    feature_store_project_name
), "Please provide a name for your Feature Store Project"

feature_store_project_exists = False
use_case = "FEATURE_STORE"
feature_store_project = None

for existing_project in client.list_projects():
    if feature_store_project_name == existing_project.name:
        feature_store_project_exists = True
        feature_store_project = existing_project

if not feature_store_project_exists:
    feature_store_project = client.create_project(feature_store_project_name, use_case)
feature_store_project

### Manipulating Data in Abacus
We can leverage Feature Groups in Abacus.ai in order to manipulate and transform our data. Each feature group must have a unique, so we define a unique prefix variable for easy uniqueness

In [ ]:
unique_prefix = ""
assert unique_prefix, "Please provide a unique_prefix to avoid name collision"

## We can make Feature Groups directly from Pandas DataFrames

In [ ]:
concrete_feature_group_name = unique_prefix + "concrete_attributes"
try:
    concrete_feature_group = client.create_feature_group_from_pandas_df(
        concrete_feature_group_name, df=concrete_df
    )
except ApiException as e:
    print(e)
    concrete_feature_group = client.describe_feature_group_by_table_name(
        concrete_feature_group_name
    )
try:
    concrete_feature_group.add_to_project(feature_store_project.id)
except ApiException as e:
    print(e)
concrete_feature_group.add_tag("base_data")
concrete_feature_group.wait_for_dataset()

## We can define generalized python functions to generate Feature Groups
First we create the function

In [ ]:
python_transform_function_name = unique_prefix + "concrete_python_transformation"
try:
    python_transform_function = client.create_python_function(
        name=python_transform_function_name
    )
except ApiException as e:
    print(e)
    python_transform_function = client.describe_python_function(
        python_transform_function_name
    )

### We then update the functions code defintion. Just like a normal python function, we have variable mappings

In [ ]:
python_transform_function = client.update_python_function_code(
    python_transform_function.name, function=transform_concrete
)
python_transform_function.function_variable_mappings

### Next, we map inputs to our bindings

In [ ]:
transform_input_feature_groups = [concrete_feature_group.table_name]
transform_function_bindings = [
    dict({"value": transform_input_feature_groups[index]}, **function_argument)
    for index, function_argument in enumerate(
        python_transform_function.function_variable_mappings
    )
]
transform_function_bindings

### Finally, we can create a feature group from this function

In [ ]:
transform_fg_name = unique_prefix + "concrete_transform_fg"
try:
    transform_fg = client.create_feature_group_from_function(
        table_name=transform_fg_name,
        python_function_name=python_transform_function.name,
        python_function_bindings=transform_function_bindings,
    )
except ApiException as e:
    print(e)
    transform_fg = client.describe_feature_group_by_table_name(transform_fg_name)
try:
    transform_fg.add_to_project(feature_store_project.id)
except ApiException as e:
    print(e)
transform_fg.add_tag("transform")
transform_fg.add_tag("python")

In [ ]:
transform_fg.id, feature_store_project.id

## We repeat the process, and can pass a python feature group as input to another python function

In [ ]:
data_split_function_name = unique_prefix + "train_test_split"
try:
    data_split_function = client.create_python_function(name=data_split_function_name)
except ApiException as e:
    print(e)
    data_split_function = client.describe_python_function(data_split_function_name)

In [ ]:
data_split_function = client.update_python_function_code(
    name=data_split_function_name, function=assign_train_test_split
)
data_split_function.function_variable_mappings

In [ ]:
split_input_feature_groups = [transform_fg.table_name]
data_split_bindings = [
    dict({"value": split_input_feature_groups[index]}, **function_argument)
    for index, function_argument in enumerate(
        data_split_function.function_variable_mappings
    )
]
data_split_bindings

In [ ]:
data_split_fg_name = unique_prefix + "concrete_data_split_fg"
try:
    data_split_fg = client.create_feature_group_from_function(
        data_split_fg_name,
        python_function_name=data_split_function.name,
        python_function_bindings=data_split_bindings,
    )
except ApiException as e:
    print(e)
    data_split_fg = client.describe_feature_group_by_table_name(data_split_fg_name)
try:
    data_split_fg.add_to_project(feature_store_project.id)
except ApiException as e:
    print(e)
data_split_fg.add_tag("data_split")
data_split_fg.add_tag("python")

In [ ]:
data_split_fg.materialize()
data_split_fg.refresh().features

# Not only Python Functions
With Feature Groups, we can also execute SQL queries. Here we split our data between train and test using SQL

In [ ]:
all_feature_but_traintest = [
    feature.name
    for feature in data_split_fg.refresh().features
    if feature.name != "TRAINTEST"
]

In [ ]:
concrete_training_data_fg_name = unique_prefix + "concrete_training_data_fg"
train_sql = f"SELECT {', '.join(all_feature_but_traintest)} FROM {data_split_fg.table_name} WHERE TRAINTEST = 'TRAIN'"
try:
    concrete_training_data_fg = client.create_feature_group(
        concrete_training_data_fg_name, sql=train_sql
    )
except ApiException as e:
    print(e)
    concrete_training_data_fg = client.describe_feature_group_by_table_name(
        concrete_training_data_fg_name
    )
try:
    concrete_training_data_fg.add_to_project(feature_store_project.id)
except ApiException as e:
    print(e)
concrete_training_data_fg.add_tag("training")
concrete_training_data_fg.add_tag("sql")

In [ ]:
concrete_predicting_data_fg_name = unique_prefix + "concrete_predicting_data_fg"
train_sql = f"SELECT {', '.join(all_feature_but_traintest)} FROM {data_split_fg.table_name} WHERE TRAINTEST = 'TEST'"
try:
    concrete_predicting_data_fg = client.create_feature_group(
        concrete_predicting_data_fg_name, sql=train_sql
    )
except ApiException as e:
    print(e)
    concrete_predicting_data_fg = client.describe_feature_group_by_table_name(
        concrete_predicting_data_fg_name
    )
try:
    concrete_predicting_data_fg.add_to_project(feature_store_project.id)
except ApiException as e:
    print(e)
concrete_predicting_data_fg.add_tag("predicting")
concrete_predicting_data_fg.add_tag("sql")

# Abacus BYOA - Run your model alongside Abacus's world class algorithms
With a handle on training a custom model in an Abacus notebook, we can now register a algorithm to run concurrently with Abacus' own algorithms. This allows us to make quick comparisons across models and helps control train and test splits.

First we create a common Predictive Modeling project

In [ ]:
byoa_project_name = ""
assert byoa_project_name, "Please provide a name for your BYOA Project"

byoa_project_exists = False
use_case = "PREDICTING"
byoa_project = None

for existing_project in client.list_projects():
    if byoa_project_name == existing_project.name:
        byoa_project_exists = True
        byoa_project = existing_project

if not byoa_project_exists:
    byoa_project = client.create_project(byoa_project_name, use_case)
byoa_project

### Add the data split feature group to the project
We also describe the requirements for the Predictive Modeling use case

In [ ]:
try:
    data_split_fg.add_to_project(byoa_project, feature_group_type="CUSTOM_TABLE")
except ApiException as e:
    print(e)

In [ ]:
client.describe_use_case_requirements(byoa_project.use_case)

### With feature mapping requirements known, we set a target

In [ ]:
byoa_project.set_feature_mapping(
    data_split_fg.id, feature_name="csMPa", feature_mapping="TARGET"
)

### Validate the project and feature groups to ensure we're ready to train a model

In [ ]:
byoa_project.validate(feature_group_ids=[data_split_fg.id])

### The client is self documenting, we can view the possible training options

In [ ]:
client.get_training_config_options(byoa_project.id, [data_split_fg.id])

## Simulate running on Abacus locally
We leverage specifically designed APIs to replicate how Abacus will pass input parameters to our training function to test locally. We can pass both Abacus and User configurations to our model

In [ ]:
training_table_names = [data_split_fg.table_name]
abacus_ai_config = {"TEST_ROW_INDICATOR": "TRAINTEST"}
user_config = {"n_quantiles": 20}
import copy

training_config = copy.deepcopy(abacus_ai_config)
training_config.update({"USER": user_config})
print(training_config)
# Change the value of the parameter names if preferred, and don't forget to change in train_func as well.
training_data_parameter_name_override = {"CUSTOM_TABLE": "training_data"}
training_config_parameter_name_override = "training_config"

### We manipulate the train function to fit the Abacus framework
And train locally

In [ ]:
def byoa_train(training_data, schema_mappings, training_config):
    import numpy as np

    train_test_column = training_config.get("TEST_ROW_INDICATOR", None)
    user_defined_config = training_config.get("USER")
    n_quantiles = user_defined_config.get("n_quantiles")

    # set the seed for reproduceable results
    np.random.seed(5)

    target_columns = schema_mappings["training_data"].get("TARGET", [])
    assert target_columns, "No target column provided"
    ignore_columns = schema_mappings["training_data"].get("IGNORE", [])
    drop_columns = target_columns + ignore_columns
    drop_columns.append(train_test_column)
    X = training_data.drop(drop_columns, axis=1).copy()
    y = training_data[target_columns]
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import QuantileTransformer

    qt = QuantileTransformer(n_quantiles=n_quantiles)
    linear_regresion = LinearRegression()
    linear_regresion.fit(qt.fit_transform(X), y)
    print(f"Linear model R^2 = {linear_regresion.score(qt.transform(X), y)}")

    return (X.columns, target_columns[0], qt, linear_regresion)

In [ ]:
# This currently returns the full data for local testing, but on remote training, only the training rows are sent to the train_functin.
train_input = client.get_train_function_input(
    project_id=byoa_project.project_id,
    training_table_names=training_table_names,
    training_data_parameter_name_override=training_data_parameter_name_override,
    training_config_parameter_name_override=training_config_parameter_name_override,
    training_config=training_config,
)
# You can also override with any dataframe for local testing.
model_input_table = data_split_fg.latest_feature_group_version.load_as_pandas()
train_input["training_data"] = model_input_table.loc[
    model_input_table["TRAINTEST"] == "TRAIN"
]
local_model = byoa_train(**train_input)

### Similarly, we define a predict many function and test

In [ ]:
def byoa_predict_many(model, queries):
    columns, target_column, qt, linear_model = model
    import pandas as pd

    result = []
    for query in queries:
        X = pd.DataFrame({c: [query[c]] for c in columns})
        y = linear_model.predict(qt.transform(X))[0]
        result.append({target_column: y[0]})
    return result

In [ ]:
prediction_results = byoa_predict_many(
    local_model, concrete_prediction_data.to_dict(orient="records")
)
prediction_results[:5]

# Registering the Algorithm
We can register an algorithm at the project or organization level, but the name must always be all caps and begin with 'USER.'

In [ ]:
algorithm_scope = "project"
algorithm_name = "USER."
assert algorithm_name.split(".")[1], "Please provide an ALLCAPS name for your algorithm"

In [ ]:
try:
    byoa_algorithm = client.create_algorithm_from_function(
        name=algorithm_name,
        problem_type="REGRESSION",
        training_data_parameter_names_mapping=training_data_parameter_name_override,
        training_config_parameter_name=training_config_parameter_name_override,
        train_function=byoa_train,
        predict_many_function=byoa_predict_many,
    )
except ApiException as e:
    print(e)
    byoa_algorithm = client.describe_algorithm(algorithm_name)
byoa_algorithm

### With a registered Algorithm, we train our model, passing Abacus and User configs

In [ ]:
import json

feature_group_ids = [
    client.describe_feature_group_by_table_name(table_name).feature_group_id
    for table_name in training_table_names
]
model_w_byoa = client.train_model(
    project_id=byoa_project.id,
    name="BYOA Regresion Model",
    training_config=abacus_ai_config,
    feature_group_ids=feature_group_ids,
    custom_algorithms=[algorithm_name],
    custom_algorithm_configs={algorithm_name: json.dumps(user_config)},
    # change to LARGE for extensive computation
    cpu_size="MEDIUM",
    # max memory is 60GB
    memory=12,
)

In [ ]:
import time
# Wait for our specific Algorithm to complete training
while algorithm_name not in [
    algo["name"]
    for algo in model_w_byoa.describe().latest_model_version.deployable_algorithms
]:
    time.sleep(5)
else:
    print(model_w_byoa.describe().latest_model_version.deployable_algorithms)

### We select our algorithm as the default in order to validate predictions
Default algorithms are utilized in our deployment unless overridden

In [ ]:
model_w_byoa.set_default_algorithm(algorithm_name)
byoa_deployment_token = client.create_deployment_token(
    byoa_project.id, "BYOA_Deployment_Token"
)
byoa_deployment = client.create_deployment(
    name="BYOA Deployment", model_id=model_w_byoa.id
)
byoa_deployment.wait_for_deployment()

### With our deployment we once again verify parity across the local predictions, the Abacus prediction API and Abacus Batch Prediction
This verifies are models are all behaving the same

In [ ]:
prediction_results = byoa_predict_many(
    local_model, concrete_prediction_data.to_dict(orient="records")
)
prediction_results[:5]

In [ ]:
import json

for record in concrete_prediction_data.to_dict(orient="records")[:5]:
    print(
        client.predict(
            deployment_token=byoa_deployment_token,
            deployment_id=byoa_deployment.id,
            query_data=json.dumps(record),
        )
    )

In [ ]:
byoa_batch_prediction = byoa_deployment.create_batch_prediction(
    table_name="byoa_evaluation_output",
    name="BYOA Evaluation Prediction",
    global_prediction_args={"forEval": True},
    output_includes_metadata=True
)
byoa_batch_prediction_run = byoa_batch_prediction.start()
byoa_batch_prediction_run.wait_for_predictions()

In [ ]:
byoa_eval_output_fg = client.describe_feature_group_by_table_name(byoa_batch_prediction.feature_group_table_name)
byoa_eval_output_fg.wait_for_materialization().latest_feature_group_version.load_as_pandas()['prediction_csMPa'][:5]

# Congratulations
You now can take a locally train a model, import it to Abacus as an independent, and import it to Abacus to run alongside our state of the art algorithms, competing against our Advanced AI systems.

In [ ]:
[{'csMPa': 37.697797557944256},
 {'csMPa': 47.571887792434},
 {'csMPa': 64.70979460520971},
 {'csMPa': 43.66334359567851},
 {'csMPa': 13.843112761580656}]